In [28]:
# Importo los paquetes
import pandas as pd
import numpy as np
import sqlite3
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier # el modelo
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix # para ver cómo funciona
from sklearn.preprocessing import LabelEncoder # poner los datos tipo int



In [29]:
# Importo los dataframes
with sqlite3.connect("../laliga.sqlite") as conn:
        df = pd.read_sql("SELECT * FROM Matches", conn)
conn.close() 

In [30]:
# Importo los dataframes
with sqlite3.connect("clasification.sqlite") as conn:
        df_teams = pd.read_sql("SELECT * FROM clasification", conn)
conn.close()  

In [31]:
# Si pongo directamente df_games =con(...) me da error en la siguiente celda
df_games=df  

In [32]:
# Le creo a df_games la columna de result (home, visitor, tie) que va a ser mi variable Y en el modelo
df_games["result"] = None

df_games.loc[(df_games["score"].str.split(":").str[0]) > (df_games["score"].str.split(":").str[1]), "result"] = 'home'
df_games.loc[(df_games["score"].str.split(":").str[0]) == (df_games["score"].str.split(":").str[1]), "result"] = 'tie'
df_games.loc[(df_games["score"].str.split(":").str[0]) < (df_games["score"].str.split(":").str[1]), "result"] = 'visitor'

In [33]:
df_games.drop(["date", "time", "score"], axis=1, inplace=True)
df_games

,season,division,matchday,home_team,away_team,result
0,1928-1929,1,1,Arenas Club,Athletic Madrid,visitor
1,1928-1929,1,1,Espanyol,Real Unión,home
2,1928-1929,1,1,Real Madrid,Catalunya,home
3,1928-1929,1,1,Donostia,Athletic,tie
4,1928-1929,1,1,Racing,Barcelona,visitor
...,...,...,...,...,...,...
48775,2021-2022,2,42,Real Oviedo,UD Ibiza,None
48776,2021-2022,2,42,Real Sociedad B,Real Zaragoza,None
48777,2021-2022,2,42,Sporting Gijón,UD Las Palmas,None
48778,2021-2022,2,42,CD Tenerife,FC Cartagena,None


In [34]:
df_games_home = df_games[["season", "division", "matchday", "home_team", "result"]]
df_games_visitor = df_games[["season", "division", "matchday", "away_team", "result"]]
df_games_home

,season,division,matchday,home_team,result
0,1928-1929,1,1,Arenas Club,visitor
1,1928-1929,1,1,Espanyol,home
2,1928-1929,1,1,Real Madrid,home
3,1928-1929,1,1,Donostia,tie
4,1928-1929,1,1,Racing,visitor
...,...,...,...,...,...
48775,2021-2022,2,42,Real Oviedo,None
48776,2021-2022,2,42,Real Sociedad B,None
48777,2021-2022,2,42,Sporting Gijón,None
48778,2021-2022,2,42,CD Tenerife,None


In [35]:
division = 1
season = "2019-2020"
matchday = 3
season_train = "2000:2010"


In [36]:
# Luego, aplicar las operaciones de cadena de caracteres
df_teams_train = df_teams

# Asegurarte de que la columna "season" del DataFrame sea de tipo cadena de caracteres
df_teams_train["season"] = df_teams_train["season"].astype(str)

df_teams_train = df_teams_train[(df_teams_train["season"].str.split("-").str[0] >= season_train.split(":")[0]) &
                    (df_teams_train["season"].str.split("-").str[1] <= season_train.split(":")[1])]
df_teams_train


,season,division,rank,team,matchday,GF,GA,GD,W,L,T,Pts
20208,2009-2010,1,1.0,Getafe,1,4.0,1.0,3.0,1.0,0.0,0.0,3.0
20209,2009-2010,1,2.0,Barcelona,1,3.0,0.0,3.0,1.0,0.0,0.0,3.0
20210,2009-2010,1,3.0,Málaga CF,1,3.0,0.0,3.0,1.0,0.0,0.0,3.0
20211,2009-2010,1,4.0,RCD Mallorca,1,2.0,0.0,2.0,1.0,0.0,0.0,3.0
20212,2009-2010,1,5.0,Valencia,1,2.0,0.0,2.0,1.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
37043,2000-2001,2,18.0,Elche CF,42,45.0,56.0,-11.0,10.0,16.0,16.0,46.0
37044,2000-2001,2,19.0,SD Compostela,42,41.0,65.0,-24.0,11.0,19.0,12.0,45.0
37045,2000-2001,2,20.0,Uni. Las Palmas,42,34.0,63.0,-29.0,8.0,19.0,15.0,39.0
37046,2000-2001,2,21.0,Getafe,42,42.0,65.0,-23.0,8.0,23.0,11.0,35.0


In [37]:
df_games_home_train = df_games_home
# Asegurarte de que la columna "season" del DataFrame sea de tipo cadena de caracteres

df_games_home_train["season"] = df_games_home_train["season"].astype(str)

# Luego, aplicar las operaciones de cadena de caracteres
df_games_home_train = df_games_home_train[(df_games_home_train["season"].str.split("-").str[0] >= season_train.split(":")[0]) &
                    (df_games_home_train["season"].str.split("-").str[1] <= season_train.split(":")[1])]
df_games_home_train


/tmp/ipykernel_713/4240325088.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_home_train["season"] = df_games_home_train["season"].astype(str)


,season,division,matchday,home_team,result
17838,2000-2001,1,1,Real Sociedad,tie
17839,2000-2001,1,1,Real Zaragoza,visitor
17840,2000-2001,1,1,Barcelona,home
17841,2000-2001,1,1,Dep. La Coruña,home
17842,2000-2001,1,1,Real Madrid,home
...,...,...,...,...,...
42851,2009-2010,2,42,Cádiz CF,home
42852,2009-2010,2,42,Celta de Vigo,visitor
42853,2009-2010,2,42,Elche CF,home
42854,2009-2010,2,42,UD Las Palmas,home


In [38]:
df_games_visitor_train = df_games_visitor
# Asegurarte de que la columna "season" del DataFrame sea de tipo cadena de caracteres

df_games_visitor_train["season"] = df_games_visitor_train["season"].astype(str)

# Luego, aplicar las operaciones de cadena de caracteres
df_games_visitor_train = df_games_visitor_train[(df_games_visitor_train["season"].str.split("-").str[0] >= season_train.split(":")[0]) &
                    (df_games_visitor_train["season"].str.split("-").str[1] <= season_train.split(":")[1])]
df_games_visitor_train

/tmp/ipykernel_713/3908699393.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_visitor_train["season"] = df_games_visitor_train["season"].astype(str)


,season,division,matchday,away_team,result
17838,2000-2001,1,1,Racing,tie
17839,2000-2001,1,1,Espanyol,visitor
17840,2000-2001,1,1,Málaga CF,home
17841,2000-2001,1,1,Athletic,home
17842,2000-2001,1,1,Valencia,home
...,...,...,...,...,...
42851,2009-2010,2,42,CD Numancia,home
42852,2009-2010,2,42,SD Huesca,visitor
42853,2009-2010,2,42,Real Sociedad,home
42854,2009-2010,2,42,Gimnàstic,home


In [39]:
df_train_home = pd.merge(df_games_home_train, df_teams_train, how="outer", 
                    left_on=['home_team', "season", "division", "matchday"],
                    right_on=['team', "season", "division", "matchday"], 
                    suffixes=("_home", ""))

df_train_home.dropna(subset="home_team", inplace=True)

In [40]:
df_train_away = pd.merge(df_games_visitor_train, df_teams_train, 
                    left_on=['away_team', "season", "division", "matchday"],
                    right_on=['team', "season", "division", "matchday"], 
                    how='outer', suffixes=("_away",""))
df_train_away.dropna(subset="away_team", inplace=True)

In [41]:
df_train = pd.concat([df_train_home, df_train_away])
df_train

,season,division,matchday,home_team,result,rank,team,GF,GA,GD,W,L,T,Pts,away_team
0,2000-2001,1,1,Real Sociedad,tie,10.0,Real Sociedad,2.0,2.0,0.0,0.0,0.0,1.0,1.0,NaN
1,2000-2001,1,1,Real Zaragoza,visitor,14.0,Real Zaragoza,1.0,2.0,-1.0,0.0,1.0,0.0,0.0,NaN
2,2000-2001,1,1,Barcelona,home,5.0,Barcelona,2.0,1.0,1.0,1.0,0.0,0.0,3.0,NaN
3,2000-2001,1,1,Dep. La Coruña,home,4.0,Dep. La Coruña,2.0,0.0,2.0,1.0,0.0,0.0,3.0,NaN
4,2000-2001,1,1,Real Madrid,home,7.0,Real Madrid,2.0,1.0,1.0,1.0,0.0,0.0,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8415,2009-2010,2,42,NaN,home,8.0,CD Numancia,55.0,53.0,2.0,16.0,15.0,11.0,59.0,CD Numancia
8416,2009-2010,2,42,NaN,visitor,13.0,SD Huesca,36.0,40.0,-4.0,12.0,14.0,16.0,52.0,SD Huesca
8417,2009-2010,2,42,NaN,home,1.0,Real Sociedad,53.0,37.0,16.0,20.0,8.0,14.0,74.0,Real Sociedad
8418,2009-2010,2,42,NaN,home,18.0,Gimnàstic,42.0,55.0,-13.0,14.0,19.0,9.0,51.0,Gimnàstic


In [42]:
# Supongamos que tienes un DataFrame df con una columna 'etiqueta' de tipo 'object'
le = LabelEncoder()
df_train['prediction'] = le.fit_transform(df_train['result'])
#df_train["home"] = le.fit_transform(df_train["home_team"])

In [43]:
# Selecciona las características (X_train) y etiquetas (y_train)
X_train = df_train[['rank', 'GD', "W", "Pts"]]  
y_train = df_train['prediction']  

In [44]:
df_teams_test = df_teams[(df_teams["season"] == season) & 
                         (df_teams["division"] == division) & 
                         (df_teams["matchday"] == matchday)]

In [45]:
df_games_home_test = df_games_home[(df_games_home["season"] == season) & 
                                   (df_games_home["division"] == division) & 
                                   (df_games_home["matchday"] == matchday)]

,season,division,matchday,home_team,result
25078,2019-2020,1,3,Sevilla FC,tie
25079,2019-2020,1,3,Athletic,home
25080,2019-2020,1,3,CA Osasuna,tie
25081,2019-2020,1,3,Getafe,tie
25082,2019-2020,1,3,Levante,home
25083,2019-2020,1,3,Real Betis,home
25084,2019-2020,1,3,Valencia,home
25085,2019-2020,1,3,Atlético Madrid,home
25086,2019-2020,1,3,Espanyol,visitor
25087,2019-2020,1,3,Villarreal,tie


In [46]:
df_games_visitor_test = df_games_visitor[(df_games_visitor["season"] == season) & 
                                   (df_games_visitor["division"] == division) & 
                                   (df_games_visitor["matchday"] == matchday)]

df_games_visitor_test


,season,division,matchday,away_team,result
25078,2019-2020,1,3,Celta de Vigo,tie
25079,2019-2020,1,3,Real Sociedad,home
25080,2019-2020,1,3,Barcelona,tie
25081,2019-2020,1,3,Alavés,tie
25082,2019-2020,1,3,Real Valladolid,home
25083,2019-2020,1,3,CD Leganés,home
25084,2019-2020,1,3,RCD Mallorca,home
25085,2019-2020,1,3,SD Eibar,home
25086,2019-2020,1,3,Granada CF,visitor
25087,2019-2020,1,3,Real Madrid,tie


In [47]:
df_test_home = pd.merge(df_games_home_test, df_teams_test, how="outer", 
                    left_on=['home_team', "season", "division", "matchday"],
                    right_on=['team', "season", "division", "matchday"], 
                    suffixes=("_home", ""))

df_test_home.dropna(subset="home_team", inplace=True)
df_test_home

,season,division,matchday,home_team,result,rank,team,GF,GA,GD,W,L,T,Pts
0,2019-2020,1,3,Sevilla FC,tie,3.0,Sevilla FC,4.0,1.0,3.0,2.0,0.0,1.0,7.0
1,2019-2020,1,3,Athletic,home,2.0,Athletic,4.0,1.0,3.0,2.0,0.0,1.0,7.0
2,2019-2020,1,3,CA Osasuna,tie,6.0,CA Osasuna,3.0,2.0,1.0,1.0,0.0,2.0,5.0
3,2019-2020,1,3,Getafe,tie,17.0,Getafe,2.0,3.0,-1.0,0.0,1.0,2.0,2.0
4,2019-2020,1,3,Levante,home,4.0,Levante,4.0,2.0,2.0,2.0,1.0,0.0,6.0
5,2019-2020,1,3,Real Betis,home,15.0,Real Betis,5.0,8.0,-3.0,1.0,2.0,0.0,3.0
6,2019-2020,1,3,Valencia,home,10.0,Valencia,3.0,2.0,1.0,1.0,1.0,1.0,4.0
7,2019-2020,1,3,Atlético Madrid,home,1.0,Atlético Madrid,5.0,2.0,3.0,3.0,0.0,0.0,9.0
8,2019-2020,1,3,Espanyol,visitor,19.0,Espanyol,0.0,5.0,-5.0,0.0,2.0,1.0,1.0
9,2019-2020,1,3,Villarreal,tie,16.0,Villarreal,7.0,8.0,-1.0,0.0,1.0,2.0,2.0


In [48]:
df_test_away = pd.merge(df_games_visitor_test, df_teams_test, 
                    left_on=['away_team', "season", "division", "matchday"],
                    right_on=['team', "season", "division", "matchday"], 
                    how='outer', suffixes=("_away",""))
df_test_away.dropna(subset="away_team", inplace=True)

In [49]:
df_test = pd.concat([df_test_home, df_test_away])
df_test

,season,division,matchday,home_team,result,rank,team,GF,GA,GD,W,L,T,Pts,away_team
0,2019-2020,1,3,Sevilla FC,tie,3.0,Sevilla FC,4.0,1.0,3.0,2.0,0.0,1.0,7.0,NaN
1,2019-2020,1,3,Athletic,home,2.0,Athletic,4.0,1.0,3.0,2.0,0.0,1.0,7.0,NaN
2,2019-2020,1,3,CA Osasuna,tie,6.0,CA Osasuna,3.0,2.0,1.0,1.0,0.0,2.0,5.0,NaN
3,2019-2020,1,3,Getafe,tie,17.0,Getafe,2.0,3.0,-1.0,0.0,1.0,2.0,2.0,NaN
4,2019-2020,1,3,Levante,home,4.0,Levante,4.0,2.0,2.0,2.0,1.0,0.0,6.0,NaN
5,2019-2020,1,3,Real Betis,home,15.0,Real Betis,5.0,8.0,-3.0,1.0,2.0,0.0,3.0,NaN
6,2019-2020,1,3,Valencia,home,10.0,Valencia,3.0,2.0,1.0,1.0,1.0,1.0,4.0,NaN
7,2019-2020,1,3,Atlético Madrid,home,1.0,Atlético Madrid,5.0,2.0,3.0,3.0,0.0,0.0,9.0,NaN
8,2019-2020,1,3,Espanyol,visitor,19.0,Espanyol,0.0,5.0,-5.0,0.0,2.0,1.0,1.0,NaN
9,2019-2020,1,3,Villarreal,tie,16.0,Villarreal,7.0,8.0,-1.0,0.0,1.0,2.0,2.0,NaN


In [50]:
le = LabelEncoder()
df_test['prediction'] = le.fit_transform(df_test['result'])
#df_test["home"] = le.fit_transform(df_test["home_team"])
df_test

,season,division,matchday,home_team,result,rank,team,GF,GA,GD,W,L,T,Pts,away_team,prediction
0,2019-2020,1,3,Sevilla FC,tie,3.0,Sevilla FC,4.0,1.0,3.0,2.0,0.0,1.0,7.0,NaN,1
1,2019-2020,1,3,Athletic,home,2.0,Athletic,4.0,1.0,3.0,2.0,0.0,1.0,7.0,NaN,0
2,2019-2020,1,3,CA Osasuna,tie,6.0,CA Osasuna,3.0,2.0,1.0,1.0,0.0,2.0,5.0,NaN,1
3,2019-2020,1,3,Getafe,tie,17.0,Getafe,2.0,3.0,-1.0,0.0,1.0,2.0,2.0,NaN,1
4,2019-2020,1,3,Levante,home,4.0,Levante,4.0,2.0,2.0,2.0,1.0,0.0,6.0,NaN,0
5,2019-2020,1,3,Real Betis,home,15.0,Real Betis,5.0,8.0,-3.0,1.0,2.0,0.0,3.0,NaN,0
6,2019-2020,1,3,Valencia,home,10.0,Valencia,3.0,2.0,1.0,1.0,1.0,1.0,4.0,NaN,0
7,2019-2020,1,3,Atlético Madrid,home,1.0,Atlético Madrid,5.0,2.0,3.0,3.0,0.0,0.0,9.0,NaN,0
8,2019-2020,1,3,Espanyol,visitor,19.0,Espanyol,0.0,5.0,-5.0,0.0,2.0,1.0,1.0,NaN,2
9,2019-2020,1,3,Villarreal,tie,16.0,Villarreal,7.0,8.0,-1.0,0.0,1.0,2.0,2.0,NaN,1


In [51]:
X_test = df_test[['rank', 'GD', "W", "Pts"]] # he cogido estás aleatoriamente
y_test = df_test['prediction']

In [52]:
# Crear el clasificador RandomForest
rf_classifier = RandomForestClassifier()

# Definir los parámetros que deseas ajustar
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Configurar la búsqueda de cuadrícula
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

# Realizar la búsqueda de cuadrícula en los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Mostrar los mejores parámetros encontrados
print("Mejores parámetros:", grid_search.best_params_)

# Obtener el modelo con los mejores parámetros
best_rf_model = grid_search.best_estimator_

Mejores parámetros: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 150}


In [53]:
# Evaluar el modelo en el conjunto de prueba
y_pred = best_rf_model.predict(X_test)
y_pred

array([0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1])

In [54]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}\n")
print(f"Confusion Matrix:\n{confusion_mat}\n")
print(f"Classification Report:\n{class_report}\n")

Accuracy: 0.75

Confusion Matrix:
[[9 1 0]
 [2 6 0]
 [1 1 0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.90      0.82        10
           1       0.75      0.75      0.75         8
           2       0.00      0.00      0.00         2

    accuracy                           0.75        20
   macro avg       0.50      0.55      0.52        20
weighted avg       0.68      0.75      0.71        20




/home/la-quiniela-main/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/la-quiniela-main/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/la-quiniela-main/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
predictions = best_rf_model.predict_proba(X_test)
predictions

array([[0.55531527, 0.2895497 , 0.15513503],
       [0.48838926, 0.36141203, 0.15019871],
       [0.15346574, 0.70713595, 0.13939831],
       [0.47364615, 0.47393622, 0.05241763],
       [0.60470282, 0.05416078, 0.3411364 ],
       [0.57198202, 0.01000314, 0.41801485],
       [0.31792558, 0.38902354, 0.29305089],
       [0.50186687, 0.13872798, 0.35940515],
       [0.50617605, 0.166375  , 0.32744894],
       [0.30978954, 0.57155496, 0.1186555 ],
       [0.44966093, 0.26820372, 0.28213536],
       [0.43138417, 0.2684787 , 0.30013713],
       [0.27554327, 0.39742728, 0.32702945],
       [0.18784803, 0.65477366, 0.1573783 ],
       [0.43748688, 0.26890162, 0.29361151],
       [0.64141066, 0.        , 0.35858934],
       [0.65016694, 0.00656879, 0.34326427],
       [0.52527585, 0.25738483, 0.21733932],
       [0.30817932, 0.38921682, 0.30260386],
       [0.25952686, 0.64694899, 0.09352415]])